### Byte Pair Encoding

In [1]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.8.0


In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces of Akwirw ier.")

In [4]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 9084, 86, 343, 86, 220, 959, 13]


In [5]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of Akwirw ier.


### Data Sampling with a Sliding Window

In [6]:
with open("data/the_verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [7]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [8]:
enc_sample = enc_text[50:]

In [9]:
context_size = 4

In [10]:
x = enc_sample[:context_size]

In [11]:
y = enc_sample[1:context_size+1]

In [12]:
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [13]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

In [15]:
text_file = "data/the_verdict.txt"

In [16]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [17]:
def create_dataloader_v1(txt, 
                         batch_size=4, 
                         max_length=256, 
                         stride=128, 
                         shuffle=True, 
                         drop_last=True, 
                         num_workers=0):
    tokenizer= tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, 
                            batch_size=batch_size, 
                            shuffle=shuffle, 
                            drop_last=drop_last, 
                            num_workers=num_workers)
    return dataloader

In [18]:
with open(text_file, "r", encoding="utf-8") as f:
    raw_text = f.read()

In [19]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

In [20]:
data_iter = iter(dataloader)

In [21]:
first_batch = next(data_iter)

In [22]:
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [23]:
second_batch = next(data_iter)

In [24]:
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


#### Increase Batch Size

In [25]:
dataloader = create_dataloader_v1( raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### Token Embeddings

In [26]:
torch.manual_seed(1969)

In [27]:
input_ids = torch.tensor([2, 3,5, 1])

In [28]:
vocab_size = 6
output_dim = 3

In [29]:
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [30]:
print(embedding_layer.weight)

Parameter containing:
tensor([[-0.0292,  0.7055,  0.7159],
        [ 2.1094, -1.2746, -1.4609],
        [ 0.1327, -0.1825, -0.9233],
        [-0.4811, -0.5903,  0.4217],
        [-0.0743, -1.1515, -0.1239],
        [ 1.0101,  0.8213,  0.6232]], requires_grad=True)


In [31]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4811, -0.5903,  0.4217]], grad_fn=<EmbeddingBackward0>)


In [32]:
print(embedding_layer(input_ids))

tensor([[ 0.1327, -0.1825, -0.9233],
        [-0.4811, -0.5903,  0.4217],
        [ 1.0101,  0.8213,  0.6232],
        [ 2.1094, -1.2746, -1.4609]], grad_fn=<EmbeddingBackward0>)


### Encoding word positions

Minor shortcoming of LLMs is that their self-attention mechanism doesn't have a notion of position or order for the tokens within a sequence. It helps to add this. Two types are absolute and relative position embeddings.

In [33]:
vocab_size = 50257
output_dim = 256

In [34]:
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [35]:
max_length = 4

In [36]:
dataloader = create_dataloader_v1(raw_text, 
                                  batch_size=8, 
                                  max_length=max_length, 
                                  stride=max_length, 
                                  shuffle=False)

In [37]:
data_iter = iter(dataloader)

In [38]:
print("Token IDs: \n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [39]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [40]:
context_length = max_length

In [41]:
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [42]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))

In [43]:
print(pos_embeddings.shape)

torch.Size([4, 256])


In [44]:
input_embeddings = token_embeddings + pos_embeddings

In [45]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])
